# LSTM Neural Network
This script has the objective of converting output of the log filter (positions) into the machine learning model using keras.

In [2]:
# Essential imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# LSTM library
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# File Management
import json

# Debug
from datetime import datetime
import time

2022-04-16 18:44:52.505357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-16 18:44:52.505456: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


First let's define the Hyper Parameters and global variables used on the algorithm.

In [32]:
PAST_TIME_STEPS = 60 # Number of past steps that will be stored on the memory

LABEL_COLUMN = 0 # Column on the object returned by the filtered log
FEATURE_COLUMNS_RANGE = (0,0)

TRAINING_PATH = "./training_model/sparkmonitor.csv"
TEST_PATH = ""

# labels definition
LABELS = ["Short Pass", "Long Pass", "Goal Shot"] 
# TODO: Complete with all labels
# OBS.: These labels will be in the matrix as index values (0..N-1),
# where N is the number of labels

Here are some functions that will be used on the code. These functions don't have initial configuration because it'll depend how the log filter will be made.

In [29]:
def get_filtered_data(path):
    """Given a path to a csv file, returns the data in numpy array format """
    # 0,0.0,0.0,0.0417605,-0.265956,1.02176,0.31014050000000004,-0.845128,1.54187,0.31014050000000004,-4.70113,1.49715,0.31014050000000004,-4.26116,-3.25787,0.3101415,-0.459378,-6.03966,0.31014050000000004,-7.03696,-2.01092,0.3101415,-6.01573,6.01449,0.31014050000000004,-7.04318,-5.98946,0.31014050000000004,-11.4279,-1.0355,0.3101335,-10.9719,-6.02464,0.3101335,-14.5318,0.00693094,0.31014050000000004,2.27424,0.0027254,0.31197050000000004,0.514697,2.30002,0.35429950000000004,0.512025,-2.33136,0.34713350000000004,4.97883,-4.99817,0.31200150000000004,4.97989,5.02929,0.35428950000000003,2.03674,-6.95322,0.31215750000000003,7.50018,-0.000507548,0.3295865,13.5358,2.93039,0.31211550000000005,14.0029,-2.94891,0.3130855,14.2025,-2.79619,0.3546115,14.4913,1.5728,0.32961450000000003

    training_csv = pd.read_csv(path, header=None).values


    print(f"{training_csv = }")

    return [] # TODO is this Json file? Or csv? Maybe csv is better


Now it's needed to get and proccess data accordingly

In [33]:
positions_training = get_filtered_data(TRAINING_PATH)
N_EXAMPLES = len(positions_training)
# TODO: convert positions_training to numpy array accordingly

# # Applying normalization
# scaler = MinMaxScaler(feature_range=(0,1)) # Apply values to [0, 1]
# positions_training_scaled = scaler.fit_transform(positions_training)

# # Convert training array to the right shape
# features_set = []
# labels = []

# for i in range(PAST_TIME_STEPS, N_EXAMPLES):
#     features_set.append(positions_training_scaled[i-PAST_TIME_STEPS:i, FEATURE_COLUMNS_RANGE[0]:FEATURE_COLUMNS_RANGE[1]]) # previous N records
#     labels.append(positions_training_scaled[i, LABEL_COLUMN])

# # Converting to numpy arrays
# features_set, labels = np.array(features_set), np.array(labels)


# # Converting to the LSTM format where the dimensions are:
# # - number of records
# # - number of time steps (past steps)
# # - number of indicators/features.
# features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], len(LABELS)))

training_csv = array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  4.1760500e-02,
        -2.6595600e-01,  1.0217600e+00,  3.1000400e-01, -8.4512800e-01,
         1.5418700e+00,  3.1000400e-01, -4.7011300e+00,  1.4971500e+00,
         3.1000040e-01, -4.2611600e+00, -3.2578700e+00,  3.1014150e-01,
        -4.5937800e-01, -6.0396600e+00,  3.1010400e-01, -7.0369600e+00,
        -2.0109200e+00,  3.1014150e-01, -6.0157300e+00,  6.0144900e+00,
         3.1004000e-01, -7.0431800e+00, -5.9894600e+00,  3.1010400e-01,
        -1.1427900e+01, -1.0355000e+00,  3.1013350e-01, -1.0971900e+01,
        -6.0246400e+00,  3.1013350e-01, -1.4531800e+01,  6.9309400e-03,
         3.1014004e-01,  2.2742400e+00,  2.7254000e-03,  3.1100400e-01,
         5.1469700e-01,  2.3000200e+00,  3.5420400e-01,  5.1202500e-01,
        -2.3313600e+00,  3.4704000e-01,  4.9788300e+00, -4.9981700e+00,
         3.1200400e-01,  4.9798900e+00,  5.0292900e+00,  3.5300000e-01,
         2.0367400e+00, -6.9532200e+00,  3.130000

With the data loaded and formatted, now it's time to create the model itself.

In [ ]:
# Model initialization
model = Sequential()

# Adding LSTM layer
# a funcao "add" literalmente adiciona um novo layer, que no nosso caso, sera um LSTM

# The fist LSTM parameter is the number of neurons on the node
# The second one is basically true, because we'll add more layers to the model
# The last one is a shape indicating the number of time steps and the number of indicators
model.add(LSTM(units=50, return_sequences=True, input_shape=(features_set.shape[1], 1)))
